In [45]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Project 3 - Interpolation and interpolating polynomials
## Part 3 - The Runge phenomenon

Laurence M. Brown
HSCI - 3316

In [46]:
# %load Makefile
# %load Makefile
#############################################################
######
# Makefile for project 3
# Laurence Brown
# SMU Mathematics
# Math 3316
# 11/2/2016
#############################################################
######
# compiler & flags
CXX = g++
CXXFLAGS = -O -std=c++0x -Wl,-undefined,dynamic_lookup
# makefile targets
#newton_test.exe	:	newton.cpp	test_newton.cpp


all:
	$(CXX) $(CXXFLAGS) test_newton.cpp -o test_newton.exe
	$(CXX) $(CXXFLAGS) Lagrange.cpp matrix.cpp test_Lagrange.cpp -o test_Langrange.exe
	$(CXX) $(CXXFLAGS) Lagrange.cpp matrix.cpp test_Lagrange2D.cpp -o test_Langrange2D.exe
	$(CXX) $(CXXFLAGS) runge_uni_cheb.cpp Lagrange.cpp matrix.cpp -o runge_uni_cheb.exe
	
	chmod 755 *.exe
	chmod 755 *.txt

clean :
	rm -f *.exe *.txt
####### End of Makefile #######


SyntaxError: invalid syntax (<ipython-input-46-00f87c7c193b>, line 13)

# The Runge Phenomenon

There is a special case, when polynomials interpolate specfic functions, that the error of the interpolating polynomial, $error = |f(x) - p_n(x)|$, does not decrease with additional data points, in fact for certain functions the maximum error will increase with the degree of the interpolating polynomial.

The Runge phenomenon can occur in the form of oscillating edges when using polynomial
interpolation with high order polynomials.

One way to avoid this problem is to change the interpolation nodes, or the data set. The Runge oscillation can be minimized by using nodes that are clusterd together as they near the ends of the interval. An example of this is Chebyshev nodes, where error of the Runge
function diminishes with additional data points.

In this report we will observe the effect on error when increasing the degree of the interpolating polynomial, and when using Chebyshev nodes as apposed to uniformly distributed nodes.

In [ ]:
# %load runge_uni_cheb.cpp
/*
	runge_uni_cheb.cpp
	Laruence M. Brown
  	Math 3316
   	3 November 2016

									Runge Phenominon 
						with uniformly and Chebyshev spaced nodes
*/

#include <string>
#include <vector>
#include <iostream>
#include <math.h>

#include "Matrix.hpp"
#include "Lagrange.hpp"

using namespace std;

//settings
#define START_GRID -4
#define END_GRID 4
#define A_LEN 201
#define B_LEN 101

//function prototypes
double 		f(double x, double y);
Matrix 		Chebyshev_nodes(int L, int m);
void 		interpolate2D(int m, int n, Matrix a, Matrix b, string output_fname, bool Chebyshev);

int main(){

	/*
		Create an array of 201 evenly-spaced evaluation points, a, over the interval [−4, 4].
		Output this to disk as the file avals.txt.

		Create an array of 101 evenly-spaced evaluation points, b, over the interval [−4, 4].
		Output this to disk as the file bvals.txt.
	*/
	Matrix as = Linspace(START_GRID, END_GRID, 1, A_LEN);
	as.Write("avals.txt");

	Matrix bs = Linspace(START_GRID, END_GRID, 1, B_LEN);
	bs.Write("bvals.txt");

	/*
		Fill a matrix runge ∈ 201×101 with the correct values of f(ai, bj), and output this to disk
		to a file named Runge.txt.
	*/
	Matrix Runge(as.Cols(), bs.Cols());
	for( int i = 0 ; i < as.Cols(); i++){
		for ( int j=0 ; j < bs.Cols(); j++)
			Runge[j][i] = f(as[i][0], bs[j][0]);
	}
	Runge.Write("Runge.txt");

	interpolate2D(6,6, as, bs, "p6_uni.txt", false);
	interpolate2D(24,24, as, bs,"p24_uni.txt", false);
	interpolate2D(6,6, as, bs,"p6_Cheb.txt", true);
	interpolate2D(24, 24, as, bs, "p24_Cheb.txt", true);

	return 0;
}

//Runge Function
double f(double x, double y){
	return 1 / (1 + x*x + y*y);
}

Matrix Chebyshev_nodes(int m){
	/*
		The (m + 1) Chebyshev nodes over an interval [−L, L] may be computed
		via the formula

		xi = Lcos ((2i + 1)π)/(2m + 2 )), i = 0 ... m
	*/
	Matrix x(1,m+1);
	for(int i =0 ;i <m+1; i++){
		x[i][0]= END_GRID * cos( (2*(i+1)*M_PI)/(2*m+2) );
	}
	return x;
}

void interpolate2D(int m, int n, Matrix a, Matrix b, string output_fname, bool Chebyshev){
	/*
		Create a set of (m + 1) evenly-spaced nodes, x, over the interval [−4, 4].
		Create a set of (n + 1) evenly-spaced nodes, x, over the interval [−4, 4].
	*/
	Matrix x = Linspace(START_GRID, END_GRID, 1, m+1);
	Matrix y = Linspace(START_GRID, END_GRID, 1, n+1);

	if (Chebyshev){
		x = Chebyshev_nodes(m);
		y = Chebyshev_nodes(n);
	}

	/*
		Create a matrix, f ∈ (m+1)×(n+1) that contains the function values f(xi,yj).
	*/
	Matrix z(m+1,n+1);
	for (int i=0; i<m+1; i++){
		for (int j=0; j<n+1; j++)
			z[i][j] = f(x[i][0], y[j][0]);
	}

	/*
		Use your Lagrange2D() function to evaluate the polynomial interpolant p(ai,bj) at
		the 201 × 101 evaluation points, storing the result in a matrix p6 ∈ 201×101.

		Output p6 to disk to a file named p6 uni.txt using the Matrix::Write routine.
	*/
	Matrix pn(a.Cols(), b.Cols());
	for (int i=0; i<a.Cols(); i++){
		for (int j=0; j<b.Cols(); j++){
			pn(i,j) = Lagrange2D(x, y, z, a[i][0], b[j][0]);
		}
	}
	pn.Write(output_fname.c_str());
}


The above code takes a function, $f(x,y) = \frac {1} {1 + x^2 + y^2}$, and creates four distinct interpolating polynomials. Two with uniformly distributed nodes over the interval $[-4,4]$ of degree 6 and 24 $(u_6(x), u_{24}(x))$. And Two with Chebyshev nodes nonuniformly distributed over the interval $[-4,4]$. The x value of each Chebyshev node is determined by the following formula: 
<br>
$$ x_i =  L_i \cos\Bigg(\frac {(2i + 1)\pi} {2m + 2}\Bigg), i = 0,1,...,m.$$
<br>
These interpolating polynomials are also of degreen 6 and 24 $(c_6(x), c_24(x))$.

In [ ]:
#load files
Runge = loadtxt('Runge.txt')
a = loadtxt('avals.txt')
b = loadtxt('bvals.txt')
p6_uni = loadtxt('p6_uni.txt')
p24_uni = loadtxt('p24_uni.txt')
p6_Cheb = loadtxt('p6_Cheb.txt')
p24_Cheb = loadtxt('p24_Cheb.txt')

## The Runge function

In [ ]:
from mpl_toolkits.mplot3d.axes3d import Axes3D
fig = figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = meshgrid(b, a)
title('$f(x)$')
surf = ax.plot_surface(X, Y, Runge, rstride=1, cstride=1, linewidth=0, cmap=cm.Greens)

The above graph is the desired value of the runge function $f(x,y) = \frac {1} {1 + x^2 + y^2}$. This function will be used to calculate the error of the interpolating polynomials graphed bellow. 

# Interpolating Polynomials with uniformly spaced nodes

## Polynomial of degree 6, with uniformly spaced nodes

In [ ]:
fig = figure()
ax = fig.add_subplot(111, projection='3d')
title('$u_6$ with uniformly spaced nodes')
surf = ax.plot_surface(X, Y, p6_uni, rstride=1, cstride=1, linewidth=0, cmap=cm.jet)

In [ ]:
#calculate error
error_6_uni = abs(Runge - p6_uni)

fig = figure()
ax = fig.add_subplot(111, projection='3d')
title('Error of $u_6(x)$ with uniformly spaced nodes')
surf = ax.plot_surface(X, Y, error_6_uni, rstride=1, cstride=1, linewidth=0, cmap=cm.Reds)

The graph produced by $u_6(x)$ looks disparate of the function it is modeling. It's error shows a sea of oscillations that become more turbulent as the ends of the intervals are approached. A low order polynomial with uniformly distributed nodes does not appear to be a good approcimation of the funtion $f(x,y) = \frac {1} {1 + x^2 + y^2}$.

## Polynomial of degree 24, with uniformly spaced nodes

In [ ]:
fig = figure()
ax = fig.add_subplot(111, projection='3d')
title('$u_{24}$ with uniformly spaced nodes')
surf = ax.plot_surface(X, Y, p24_uni, rstride=1, cstride=1, linewidth=0, cmap=cm.jet)

In [ ]:
#calculate error
error_24_uni = abs(Runge - p24_uni)

fig = figure()
ax = fig.add_subplot(111, projection='3d')
title('Error of $u_{24}(x)$ with uniformly spaced nodes')
surf = ax.plot_surface(X, Y, error_24_uni, rstride=1, cstride=1, linewidth=0, cmap=cm.Reds)

Moving on to a higher order polynomial we find intresting results, and the worst error of all the interpolating polynomials. The graph produced by $u_12(x)$ appears to have little to no error in the majority of the xy plane. However, as the ends of the intervals are approached the error becomes unbelivable high. A high order polynomial with uniformly distributed nodes does not appear to be a good approcimation of the funtion $f(x,y) = \frac {1} {1 + x^2 + y^2}$, at least at the edges of its intervals.

# Interpolating Polynomials with Chebyshev nodes

## Polynomial of degree 6, with Chebyshev nodes

In [ ]:
fig = figure()
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, p6_Cheb, rstride=1, cstride=1, linewidth=0, cmap=cm.jet)
title('$c_6$ with Chebyshev nodes')

In [ ]:
#calculate error
error_6_cheb = abs(Runge - p6_Cheb)

fig = figure()
ax = fig.add_subplot(111, projection='3d')
title('Error of $c_6(x)$ with chebyshev nodes')
surf = ax.plot_surface(X, Y, error_6_cheb, rstride=1, cstride=1, linewidth=0, cmap=cm.Reds)

Taking a look at the graph of a low order polynomial with Chebyshev nodes. we find intresting results. The graph produced by $c_6(x)$ is the first graph to resemble $f(x,y) = \frac {1} {1 + x^2 + y^2}$. However, error remians. Intrestingly it is not at the corners of the graph that the maximum error peaks (as with the errors produced by the unifromly distributed polynomials), but at the center of the xy plane. The low order polynomial with Chebyshev nodes is the best approximation of the funtion $f(x,y) = \frac {1} {1 + x^2 + y^2}$, thus far.

## Polynomial of degree 24, with Chebyshev nodes

In [ ]:
fig = figure()
ax = fig.add_subplot(111, projection='3d')
title('$c_{24}$ with Chebyshev nodes')
surf = ax.plot_surface(X, Y, p24_Cheb, cmap=cm.jet)

In [ ]:
#calculate error
error_24_cheb = abs(Runge - p24_Cheb)

fig = figure()
ax = fig.add_subplot(111, projection='3d')
title('Error of $c_{24}(x)$ with chebyshev nodes')
surf = ax.plot_surface(X, Y, error_24_cheb, rstride=1, cstride=1, linewidth=0, cmap=cm.Reds)

The graph produced by the high order polynomial with Chebyshev nodes, $c_{24}(x)$ is the best approximation of the function $f(x,y) = \frac {1} {1 + x^2 + y^2}$. The graph produced by $c_{24}(x)$ looks like the one produced by $f(x)$, it is only when observing the error that discrepcenies are noticed. It seems that with Chebyshev nodes, as perdicted, error of the Runge function decreases with additional data points. This is unlike the uniformly distributed notes wich display the oposite behavior.